In [3]:
import numpy as np
# import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from numpy import random

In [4]:
from quad_model import Quadrotor
from mpc_setup import MPC

# Set up Data Generation


In [5]:
def get_outputs(num_data_gen=100):
  allData = np.empty((0, 22))
  sim = True
  Thorizon = 2.0
  Nhorizon = 50
  use_cython = False
  mpc_solver = MPC(use_cython=use_cython, sim=sim, quadrotor=Quadrotor(sim), T_horizon=Thorizon, N_horizon=Nhorizon)
  
  # print(allData)
  for i in range(num_data_gen):
    curr_x, curr_y, curr_z = random.uniform(-3,3), random.uniform(-3,3), random.uniform(0,-2)
    curr_vx, curr_vy, curr_vz = random.uniform(-.7,.7), random.uniform(-.7,.7), random.uniform(-.7,.7)
    curr_roll, curr_pitch, curr_yaw = random.uniform(-.2,.2), random.uniform(-.2,.2), random.uniform(-.2,.2)
    x0 = np.array([curr_x, curr_y, curr_z, curr_vx, curr_vy, curr_vz, curr_roll, curr_pitch, curr_yaw])
    # print(x0.shape)

    # ref_x, ref_y, ref_z = random.uniform(-3,3), random.uniform(-3,3), random.uniform(0,-2)
    # ref_vx, ref_vy, ref_vz = random.uniform(-.7,.7), random.uniform(-.7,.7), random.uniform(-.7,.7)
    # ref_roll, ref_pitch, ref_yaw = random.uniform(-.2,.2), random.uniform(-.2,.2), random.uniform(-.2,.2)
    # xref = np.array([ref_x, ref_y, ref_z, ref_vx, ref_vy, ref_vz, ref_roll, ref_pitch, ref_yaw])
    # Define perturbation ranges

    # These ranges define how far xref can be from x0 for each component
    perturbation_ranges = np.array([
        0.5,  # x
        0.5,  # y
        0.2,  # z
        0.1,  # vx
        0.1,  # vy
        0.1,  # vz
        0.05, # roll
        0.05, # pitch
        0.05  # yaw
    ])
    # Generate xref by adding a random perturbation to x0
    # Perturbation is chosen randomly within the specified range
    perturbations = perturbation_ranges * (2 * np.random.rand(9) - 1)  # random values in [-1, 1] scaled by perturbation_ranges
    xref = x0 + perturbations
    # print(xref.shape)


    u = mpc_solver.solve_mpc(x0, xref)
    # print(u)
    # print(x0)
    # print(xref)
    # print(u.tolist())
  

    # outputs2 = outputs.squeeze().tolist()
    finalData = x0.tolist() + xref.tolist() + u.tolist()
    # print(len(finalData))
    finalData = np.array(finalData, dtype=float)
    allData = np.vstack((allData, finalData))

  return allData

In [6]:
# data_all = get_outputs(50_000)
# 

In [7]:
50_000/128

390.625

In [8]:
data_all[0]

NameError: name 'data_all' is not defined

In [9]:
# np.save('50k_MPC_data', data_all)


# Set up DEQ

### First Get My Data and Set Up DataLoaders

In [10]:
data_all = np.load('50k_MPC_data.npy')
data_all.shape

(50000, 22)

In [11]:
class QuadrotorDataset(Dataset):
  def __init__(self, statevec_size = 9, ctrlvec_size = 4, training=True, data_all = data_all):
    self.input_size = 2*statevec_size
    self.output_size = ctrlvec_size

    self.data = data_all

    np.random.shuffle(data_all)
    train_size = int(len(data_all)*0.8)

    if training:
      self.data = data_all[0:train_size]
    else:
      self.data = data_all[train_size:]

    print(self.data.shape)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    inputs = self.data[idx, 0: self.input_size]
    outputs = self.data[idx, self.input_size:]
    # print(type(inputs))
    # print(type(outputs))
    # print(inputs)
    # print(inputs.dtype)
    # print("h")
    return (torch.FloatTensor(inputs), torch.FloatTensor(outputs))

In [12]:
batch_size = 128
train_dataset = QuadrotorDataset(training=True)
test_dataset = QuadrotorDataset(training=False)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
)


(40000, 22)
(10000, 22)


In [13]:
# print(train_dataset[0])
# print(test_dataset[0])
train_dataset[0]


(tensor([-1.1562,  1.3734, -1.6282, -0.5585,  0.6041, -0.2989, -0.1755, -0.1391,
          0.1769, -1.2238,  1.5272, -1.4971, -0.5912,  0.5936, -0.2370, -0.1631,
         -0.1768,  0.1937]),
 tensor([14.5329,  0.0993,  0.0804,  0.0176]))

## Necessary Functions, Classes, and Imports for Building and Training INN


### Imports

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import time
from torch.autograd import Function


### MonSingleFC and MONReLU

In [15]:
class MONSingleFc(nn.Module):
    """ Simple MON linear class, just a single full multiply. """

    def __init__(self, in_dim, out_dim, m=1.0):
        super().__init__()
        self.U = nn.Linear(in_dim, out_dim)
        self.A = nn.Linear(out_dim, out_dim, bias=False)
        self.B = nn.Linear(out_dim, out_dim, bias=False)
        self.m = m

    def x_shape(self, n_batch):
        return (n_batch, self.U.in_features)

    def z_shape(self, n_batch):
        return ((n_batch, self.A.in_features),)

    def forward(self, x, *z):
        return (self.U(x) + self.multiply(*z)[0],)

    def bias(self, x):
        return (self.U(x),)

    def multiply(self, *z):
        ATAz = self.A(z[0]) @ self.A.weight
        z_out = (1 - self.m) * z[0] - ATAz + self.B(z[0]) - z[0] @ self.B.weight
        return (z_out,)

    def multiply_transpose(self, *g):
        ATAg = self.A(g[0]) @ self.A.weight
        g_out = (1 - self.m) * g[0] - ATAg - self.B(g[0]) + g[0] @ self.B.weight
        return (g_out,)

    def init_inverse(self, alpha, beta):
        I = torch.eye(self.A.weight.shape[0], dtype=self.A.weight.dtype,
                      device=self.A.weight.device)
        W = (1 - self.m) * I - self.A.weight.T @ self.A.weight + self.B.weight - self.B.weight.T
        self.Winv = torch.inverse(alpha * I + beta * W)

    def inverse(self, *z):
        return (z[0] @ self.Winv.transpose(0, 1),)

    def inverse_transpose(self, *g):
        return (g[0] @ self.Winv,)


class MONReLU(nn.Module):
    def forward(self, *z):
        return tuple(F.relu(z_) for z_ in z)

    def derivative(self, *z):
        return tuple((z_ > 0).type_as(z[0]) for z_ in z)


### Splitting


#### 1.Utils


In [16]:
class Meter(object):
    """Computes and stores the min, max, avg, and current values"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.max = -float("inf")
        self.min = float("inf")

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.max = max(self.max, val)
        self.min = min(self.min, val)


class SplittingMethodStats(object):
    def __init__(self):
        self.fwd_iters = Meter()
        self.bkwd_iters = Meter()
        self.fwd_time = Meter()
        self.bkwd_time = Meter()

    def reset(self):
        self.fwd_iters.reset()
        self.fwd_time.reset()
        self.bkwd_iters.reset()
        self.bkwd_time.reset()

    def report(self):
        print('Fwd iters: {:.2f}\tFwd Time: {:.4f}\tBkwd Iters: {:.2f}\tBkwd Time: {:.4f}\n'.format(
                self.fwd_iters.avg, self.fwd_time.avg,
                self.bkwd_iters.avg, self.bkwd_time.avg))


#### 2.Splitting Files


In [17]:
class MONPeacemanRachford(nn.Module):

    def __init__(self, linear_module, nonlin_module, alpha=1.0, tol=1e-5, max_iter=50, verbose=False):
        super().__init__()
        self.linear_module = linear_module
        self.nonlin_module = nonlin_module
        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter
        self.verbose = verbose
        self.stats = SplittingMethodStats()
        self.save_abs_err = False

    def forward(self, x):
        """ Forward pass of the MON, find an equilibirum with forward-backward splitting"""

        start = time.time()
        # Run the forward pass _without_ tracking gradients
        self.linear_module.init_inverse(1 + self.alpha, -self.alpha)
        with torch.no_grad():
            z = tuple(torch.zeros(s, dtype=x.dtype, device=x.device)
                      for s in self.linear_module.z_shape(x.shape[0]))
            u = tuple(torch.zeros(s, dtype=x.dtype, device=x.device)
                      for s in self.linear_module.z_shape(x.shape[0]))

            n = len(z)
            bias = self.linear_module.bias(x)

            err = 1.0
            it = 0
            errs = []
            while (err > self.tol and it < self.max_iter):
                u_12 = tuple(2 * z[i] - u[i] for i in range(n))
                z_12 = self.linear_module.inverse(*tuple(u_12[i] + self.alpha * bias[i] for i in range(n)))
                u = tuple(2 * z_12[i] - u_12[i] for i in range(n))
                zn = self.nonlin_module(*u)

                if self.save_abs_err:
                    fn = self.nonlin_module(*self.linear_module(x, *zn))
                    err = sum((zn[i] - fn[i]).norm().item() / (zn[i].norm().item()) for i in range(n))
                    errs.append(err)
                else:
                    err = sum((zn[i] - z[i]).norm().item() / (1e-6 + zn[i].norm().item()) for i in range(n))
                z = zn
                it = it + 1

        if self.verbose:
            print("Forward: ", it, err)

        # Run the forward pass one more time, tracking gradients, then backward placeholder
        zn = self.linear_module(x, *z)
        zn = self.nonlin_module(*zn)

        zn = self.Backward.apply(self, *zn)
        self.stats.fwd_iters.update(it)
        self.stats.fwd_time.update(time.time() - start)
        self.errs = errs
        return zn

    class Backward(Function):
        @staticmethod
        def forward(ctx, splitter, *z):
            ctx.splitter = splitter
            ctx.save_for_backward(*z)
            return z

        @staticmethod
        def backward(ctx, *g):
            start = time.time()
            sp = ctx.splitter
            n = len(g)
            z = ctx.saved_tensors
            j = sp.nonlin_module.derivative(*z)
            I = [j[i] == 0 for i in range(n)]
            d = [(1 - j[i]) / j[i] for i in range(n)]
            v = tuple(j[i] * g[i] for i in range(n))

            z = tuple(torch.zeros(s, dtype=g[0].dtype, device=g[0].device)
                      for s in sp.linear_module.z_shape(g[0].shape[0]))
            u = tuple(torch.zeros(s, dtype=g[0].dtype, device=g[0].device)
                      for s in sp.linear_module.z_shape(g[0].shape[0]))

            err = 1.0
            errs=[]
            it = 0
            while (err >sp.tol and it < sp.max_iter):
                u_12 = tuple(2 * z[i] - u[i] for i in range(n))
                z_12 = sp.linear_module.inverse_transpose(*u_12)
                u = tuple(2 * z_12[i] - u_12[i] for i in range(n))
                zn = tuple((u[i] + sp.alpha * (1 + d[i]) * v[i]) / (1 + sp.alpha * d[i]) for i in range(n))
                for i in range(n):
                    zn[i][I[i]] = v[i][I[i]]

                err = sum((zn[i] - z[i]).norm().item() / (1e-6 + zn[i].norm().item()) for i in range(n))
                errs.append(err)
                z = zn
                it = it + 1

            if sp.verbose:
                print("Backward: ", it, err)

            dg = sp.linear_module.multiply_transpose(*zn)
            dg = tuple(g[i] + dg[i] for i in range(n))

            sp.stats.bkwd_iters.update(it)
            sp.stats.bkwd_time.update(time.time() - start)
            sp.errs = errs
            return (None,) + dg

### SingleFC Net and Support Functions


In [18]:
def expand_args(defaults, kwargs):
    d = defaults.copy()
    for k, v in kwargs.items():
        d[k] = v
    return d

MON_DEFAULTS = {
    'alpha': 1.0,
    'tol': 1e-5,
    'max_iter': 50
}

class SingleFcNet(nn.Module):

    def __init__(self, splittingMethod, in_dim=18, intermediate_dim=25, out_dim=4, m=0.1, **kwargs):
        super().__init__()
        linear_module = MONSingleFc(in_dim, intermediate_dim, m=m)
        nonlin_module = MONReLU()
        self.mon = splittingMethod(linear_module, nonlin_module, **expand_args(MON_DEFAULTS, kwargs))
        self.Wout = nn.Linear(intermediate_dim, out_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        z = self.mon(x)
        return self.Wout(z[-1])

### Training Function


In [19]:
def cuda(tensor):
    if torch.cuda.is_available():
        return tensor.cuda()
    else:
        return tensor
def run_tune_alpha(model, x, max_alpha):
    print("----tuning alpha----")
    print("current: ", model.mon.alpha)
    orig_alpha  =  model.mon.alpha
    model.mon.stats.reset()
    model.mon.alpha = max_alpha
    with torch.no_grad():
        model(x)
    iters = model.mon.stats.fwd_iters.val
    model.mon.stats.reset()
    iters_n = iters
    print('alpha: {}\t iters: {}'.format(model.mon.alpha, iters_n))
    while model.mon.alpha > 1e-4 and iters_n <= iters:
        model.mon.alpha = model.mon.alpha/2
        with torch.no_grad():
            model(x)
        iters = iters_n
        iters_n = model.mon.stats.fwd_iters.val
        print('alpha: {}\t iters: {}'.format(model.mon.alpha, iters_n))
        model.mon.stats.reset()

    if iters==model.mon.max_iter:
        print("none converged, resetting to current")
        model.mon.alpha=orig_alpha
    else:
        model.mon.alpha = model.mon.alpha * 2
        print("setting to: ", model.mon.alpha)
    print("--------------\n")

def train(trainLoader, testLoader, model, epochs=15, max_lr=1e-3,
          print_freq=10, change_mo=True, model_path=None, lr_mode='step',
          step=10,tune_alpha=False,max_alpha=1.):


    optimizer = optim.Adam(model.parameters(), lr=max_lr)

    if lr_mode == '1cycle':
        lr_schedule = lambda t: np.interp([t],
                                          [0, (epochs-5)//2, epochs-5, epochs],
                                          [1e-3, max_lr, 1e-3, 1e-3])[0]
    elif lr_mode == 'step':
        lr_scheduler =optim.lr_scheduler.StepLR(optimizer, step, gamma=0.1, last_epoch=-1)
    elif lr_mode != 'constant':
        raise Exception('lr mode one of constant, step, 1cycle')

    if change_mo:
        max_mo = 0.85
        momentum_schedule = lambda t: np.interp([t],
                                                [0, (epochs - 5) // 2, epochs - 5, epochs],
                                                [0.95, max_mo, 0.95, 0.95])[0]

    model = cuda(model)

    for epoch in range(1, 1 + epochs):
        nProcessed = 0
        nTrain = len(trainLoader.dataset)
        model.train()
        start = time.time()
        for batch_idx, batch in enumerate(trainLoader):
            if (batch_idx  == 30 or batch_idx == int(len(trainLoader)/2)) and tune_alpha:
                run_tune_alpha(model, cuda(batch[0]), max_alpha)
            if lr_mode == '1cycle':
                lr = lr_schedule(epoch -  1 + batch_idx/ len(trainLoader))
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
            if change_mo:
                beta1 = momentum_schedule(epoch - 1 + batch_idx / len(trainLoader))
                for param_group in optimizer.param_groups:
                    param_group['betas'] = (beta1, optimizer.param_groups[0]['betas'][1])

            data, target = cuda(batch[0]), cuda(batch[1])
            # print("HEREEEEEEE2")
            optimizer.zero_grad()
            preds = model(data)
            # print(data.shape)
            # print(preds.shape)
            # print(target.shape)
            ce_loss = nn.MSELoss()(preds, target)
            ce_loss.backward()
            nProcessed += len(data)
            if batch_idx % print_freq == 0 and batch_idx > 0:
            #     incorrect = preds.float().argmax(1).ne(target.data).sum()
            #     err = 100. * incorrect.float() / float(len(data))
                partialEpoch = epoch + batch_idx / len(trainLoader) - 1
                print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.4f}'.format(
                    partialEpoch, nProcessed, nTrain,
                    100. * batch_idx / len(trainLoader),
                    ce_loss.item()))
                model.mon.stats.report()
                model.mon.stats.reset()
            optimizer.step()

        if lr_mode == 'step':
            lr_scheduler.step()

        if model_path is not None:
            torch.save(model.state_dict(), model_path)

        print("Tot train time: {}".format(time.time() - start))

        start = time.time()
        test_loss = 0
        incorrect = 0
        model.eval()
        with torch.no_grad():
            for batch in testLoader:
                data, target = cuda(batch[0]), cuda(batch[1])
                preds = model(data)
                ce_loss = nn.MSELoss(reduction='sum')(preds, target)
                test_loss += ce_loss
                # incorrect += preds.float().argmax(1).ne(target.data).sum()
            test_loss /= len(testLoader.dataset)
            # nTotal = len(testLoader.dataset)
            # err = 100. * incorrect.float() / float(nTotal)
            print('\n\nTest set: Test loss: {:.4f}'.format(
                test_loss))

        print("Tot test time: {}\n\n\n\n".format(time.time() - start))


### Create Model


In [24]:
model_DEQ = SingleFcNet(MONPeacemanRachford,
                        in_dim = 18,
                        intermediate_dim = 100,
                        out_dim = 4,
                        alpha=1.0,
                        max_iter=300,
                        tol=1e-2,
                        m=1.0)

# Train It!

In [25]:
trainLoader, testLoader = train_dataloader, test_dataloader

train(trainLoader, testLoader,
      model_DEQ,
      max_lr=1e-3,
      lr_mode='step',
      step=10,
      change_mo=False,
      epochs=100,
    #   epochs=1,
      print_freq=100,
      tune_alpha=True)

----tuning alpha----
current:  1.0
alpha: 1.0	 iters: 7
alpha: 0.5	 iters: 6
alpha: 0.25	 iters: 8
setting to:  0.5
--------------

Train Epoch: 0.32 [12928/40000 (32%)]	Loss: 17.5688
Fwd iters: 6.66	Fwd Time: 0.0018	Bkwd Iters: 7.00	Bkwd Time: 0.0034

----tuning alpha----
current:  0.5
alpha: 1.0	 iters: 9
alpha: 0.5	 iters: 7
alpha: 0.25	 iters: 9
setting to:  0.5
--------------

Train Epoch: 0.64 [25728/40000 (64%)]	Loss: 2.6957
Fwd iters: 7.00	Fwd Time: 0.0018	Bkwd Iters: 7.00	Bkwd Time: 0.0032

Train Epoch: 0.96 [38528/40000 (96%)]	Loss: 3.1495
Fwd iters: 7.00	Fwd Time: 0.0019	Bkwd Iters: 7.00	Bkwd Time: 0.0033

Tot train time: 2.9261600971221924


Test set: Test loss: 10.5840
Tot test time: 0.36722540855407715




----tuning alpha----
current:  0.5
alpha: 1.0	 iters: 9
alpha: 0.5	 iters: 7
alpha: 0.25	 iters: 9
setting to:  0.5
--------------

Train Epoch: 1.32 [12928/40000 (32%)]	Loss: 1.9820
Fwd iters: 7.00	Fwd Time: 0.0019	Bkwd Iters: 7.00	Bkwd Time: 0.0034

----tuning alpha--

# Test It!

In [117]:
i = 0
bad = 0
num_trials = 1000
for data in train_dataloader:
    input_data, output_data = data
    # print(input_data[0])
    # print(output_data[0])
    output_pred = model_DEQ(input_data)
    # print(output_pred[0])
    if ((output_pred[0] - output_data[0]) < torch.tensor([.08]*4)).all():
        pass
        # print("Good Prediction!")
    else:
        bad +=1
        # print("Bad Prediction")

    i += 1
    if i >= 100:
        print(f"error pct: {(bad/num_trials)*100}%") #.8% is the best I've seen with 100intermediate dim and 100 epochs
        break

error pct: 5.7%


In [24]:
trainable_params = sum(
	p.numel() for p in model_DEQ.parameters() if p.requires_grad
)
trainable_params

6154

In [119]:
total_params = sum(
	param.numel() for param in model_DEQ.parameters()
)
total_params

6154

In [120]:
model_DEQ.parameters

<bound method Module.parameters of SingleFcNet(
  (mon): MONPeacemanRachford(
    (linear_module): MONSingleFc(
      (U): Linear(in_features=18, out_features=50, bias=True)
      (A): Linear(in_features=50, out_features=50, bias=False)
      (B): Linear(in_features=50, out_features=50, bias=False)
    )
    (nonlin_module): MONReLU()
  )
  (Wout): Linear(in_features=50, out_features=4, bias=True)
)>

In [121]:
input = torch.tensor([-1.6059, -2.3186, -1.1661, -0.1998, -0.3612, -0.1757,  0.1967, -0.0053,
          0.1426, -1.9095, -2.3642, -1.1908, -0.1444, -0.3025, -0.0762,  0.1597,
         -0.0116,  0.1635])

In [122]:
input = input.unsqueeze(0)
input.shape


torch.Size([1, 18])

In [123]:
output = model_DEQ(input)

In [124]:
output

tensor([[ 1.4836e+01, -4.0354e-01,  1.2055e-01,  1.6976e-03]],
       grad_fn=<AddmmBackward0>)

# Save the Model!

In [116]:
SAVE_PATH = "DEQ_MPCmodel50k_50intermediate_100epoch.pt"
torch.save(model_DEQ.state_dict(), SAVE_PATH)

# Test A Pretrained INN

In [16]:
loaded_model = SingleFcNet(MONPeacemanRachford,
                        in_dim = 18,
                        out_dim = 4,
                        alpha=1.0,
                        max_iter=300,
                        tol=1e-2,
                        m=1.0)

In [17]:
LOAD_PATH = "DEQ_MPCmodel50k.pt"
loaded_model.load_state_dict(torch.load(LOAD_PATH))

<All keys matched successfully>

In [20]:
trainable_params = sum(
	p.numel() for p in loaded_model.parameters() if p.requires_grad
)
trainable_params

22304

In [22]:
loaded_model.parameters

<bound method Module.parameters of SingleFcNet(
  (mon): MONPeacemanRachford(
    (linear_module): MONSingleFc(
      (U): Linear(in_features=18, out_features=100, bias=True)
      (A): Linear(in_features=100, out_features=100, bias=False)
      (B): Linear(in_features=100, out_features=100, bias=False)
    )
    (nonlin_module): MONReLU()
  )
  (Wout): Linear(in_features=100, out_features=4, bias=True)
)>

In [88]:
i = 0
bad = 0
num_trials = 1000
for data in train_dataloader:
    input_data, output_data = data
    # print(input_data[0])
    # print(output_data[0])
    output_pred = loaded_model(input_data)
    # print(output_pred[0])
    if ((output_pred[0] - output_data[0]) < torch.tensor([.08]*4)).all():
        pass
        # print("Good Prediction!")
    else:
        bad +=1
        # print("Bad Prediction")

    i += 1
    if i >= 100:
        print(f"error pct: {(bad/num_trials)*100}%")
        break

error pct: 0.5%


In [89]:
new_input = input_data
output = loaded_model(new_input)

In [90]:
output

tensor([[ 1.4558e+01, -5.5158e-01,  6.9803e-01,  2.0934e-02],
        [ 1.4284e+01, -5.8372e-01,  1.5317e-01,  2.5832e-04],
        [ 1.5265e+01,  7.7864e-01, -8.2178e-01, -4.3077e-02],
        [ 1.4017e+01, -7.5778e-01,  3.7752e-01,  2.3120e-02],
        [ 1.5233e+01,  3.9309e-01, -3.1720e-01,  1.4570e-02],
        [ 1.4868e+01,  1.5233e-01,  7.7976e-01, -1.5642e-02],
        [ 1.5577e+01,  6.1416e-01,  2.5127e-01, -6.4853e-02],
        [ 1.6279e+01, -4.1952e-01,  5.5970e-01, -1.0195e-02],
        [ 1.4562e+01,  5.8141e-01,  7.6463e-01,  2.6062e-02],
        [ 1.5220e+01,  5.5413e-01, -7.8391e-01,  8.9096e-02],
        [ 1.5440e+01,  6.2369e-01, -8.1555e-01,  9.8543e-02],
        [ 1.4938e+01,  7.8784e-01,  2.8896e-01,  2.7204e-03],
        [ 1.4969e+01, -6.4334e-01,  1.0049e-01, -1.1519e-02],
        [ 1.5629e+01,  5.0254e-01, -8.2082e-01, -2.8764e-02],
        [ 1.5858e+01, -7.5084e-01,  8.2138e-01,  5.3770e-02],
        [ 1.5249e+01,  8.0691e-01, -7.9420e-01,  5.4668e-02],
        

In [91]:
input = torch.tensor([-1.6059, -2.3186, -1.1661, -0.1998, -0.3612, -0.1757,  0.1967, -0.0053,
          0.1426, -1.9095, -2.3642, -1.1908, -0.1444, -0.3025, -0.0762,  0.1597,
         -0.0116,  0.1635])



In [92]:
input = input.unsqueeze(0)
input.shape

torch.Size([1, 18])

In [93]:
output = loaded_model(input)

In [94]:
output

tensor([[14.8450, -0.4546,  0.0920,  0.0302]], grad_fn=<AddmmBackward0>)

# Actual reference functions for possible future use

In [ ]:
# ~~ The following functions are reference trajectories for tracking ~~
def hover_ref_func(self, num): #Returns Constant Hover Reference Trajectories At A Few Different Positions for Testing ([x,y,z,yaw])
    hover_dict = {
        # 1: np.array([[0.0, 0.0, -0.5, 0.0]]).T,
        1: np.array([[0.0, 0.0, -0.5,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        2: np.array([[0.0, 1.5, -1.5,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        3: np.array([[1.5, 0.0, -1.5,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        4: np.array([[1.5, 1.5, -1.5,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        5: np.array([[0.0, 0.0, -10.0,    0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        6: np.array([[1.0, 1.0, -4.0,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        7: np.array([[3.0, 4.0, -5.0,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
        8: np.array([[1.0, 1.0, -3.0,     0.0, 0.0, 0.0,   0.0, 0.0, 0.0]]).T,
    }
    if num > len(hover_dict) or num < 1:
        print(f"hover1- #{num} not found")
        exit(0)
        # return np.array([[0.0, 0.0, 0.0, self.yaw]]).T

    if not self.sim:
        if num > 4:
            print("hover modes 5+ not available for hardware")
            exit(0)
            # return np.array([[0.0, 0.0, 0.0, self.yaw]]).T
        
    print(f"hover1- #{num}")
    r = hover_dict.get(num)
    # r_final = np.tile(r, (1, self.num_steps))
    return r


def circle_vert_ref_func(self): #Returns Circle Reference Trajectory in Vertical Plane ([x,y,z,yaw])
    print("circle_vert_ref_func")

    t = self.timefromstart
    w=1;

    x = 0.0
    y = .4 * np.cos(w*t)
    z = -1*(.4*np.sin(w*t)+1.5)
    vx = 0.0
    vy = 0.0
    vz = 0.0
    roll = 0.0
    pitch = 0.0
    yaw = 0.0

    r = np.array([[x, y, z, vx, vy, vz, roll, pitch, yaw]]).T
    # r_final = np.tile(r, (1, self.num_steps))
    return r

def circle_horz_ref_func(self): #Returns Circle Reference Trajectory in Horizontal Plane ([x,y,z,yaw])
    print("circle_horz_ref_func")

    t = self.timefromstart
    w=1;
    # r = np.array([[.8*np.cos(w*t), .8*np.sin(w*t), -1.25, 0.0]]).T

    x = .8*np.cos(w*t)
    y = .8*np.sin(w*t)
    z = -1.25
    vx = 0.0
    vy = 0.0
    vz = 0.0
    roll = 0.0
    pitch = 0.0
    yaw = 0.0

    r = np.array([[x, y, z, vx, vy, vz, roll, pitch, yaw]]).T
    # r_final = np.tile(r, (1, self.num_steps))

    return r

def fig8_horz_ref_func(self): #Returns Figure 8 Reference Trajectory in Horizontal Plane ([x,y,z,yaw])
    print("fig8_horz_ref_func")

    t = self.timefromstart
    # r = np.array([[.35*np.sin(2*t/2), .35*np.sin(t/2), -1.25, 0.0]]).T

    x = .35*np.sin(2*t/2)
    y = .35*np.sin(t/2)
    z = -1.25
    vx = 0.0
    vy = 0.0
    vz = 0.0
    roll = 0.0
    pitch = 0.0
    yaw = 0.0

    r = np.array([[x, y, z, vx, vy, vz, roll, pitch, yaw]]).T
    # r_final = np.tile(r, (1, self.num_steps))

    return r

def fig8_vert_ref_func_short(self): #Returns A Short Figure 8 Reference Trajectory in Vertical Plane ([x,y,z,yaw])
    print(f"fig8_vert_ref_func_short")

    t = self.timefromstart
    # r = np.array([[0.0, .4*np.sin(t), -1*(.4*np.sin(2*t) + 1.25), 0.0]]).T

    x = 0.0
    y = .4*np.sin(t)
    z = -1*(.4*np.sin(2*t)+1.25)
    vx = 0.0
    vy = 0.0
    vz = 0.0
    roll = 0.0
    pitch = 0.0
    yaw = 0.0

    r = np.array([[x, y, z, vx, vy, vz, roll, pitch, yaw]]).T
    # r_final = np.tile(r, (1, self.num_steps))

    return r

def fig8_vert_ref_func_tall(self): #Returns A Tall Figure 8 Reference Trajectory in Vertical Plane ([x,y,z,yaw])
    print(f"fig8_vert_ref_func_tall")

    t = self.timefromstart
    # r = np.array([[0.0, .4*np.sin(2*t), -1*(.4*np.sin(t)+1.25), 0.0]]).T

    x = 0.0
    y = .4*np.sin(2*t)
    z = -1*(.4*np.sin(t)+1.25)
    vx = 0.0
    vy = 0.0
    vz = 0.0
    roll = 0.0
    pitch = 0.0
    yaw = 0.0

    r = np.array([[x, y, z, vx, vy, vz, roll, pitch, yaw]]).T
    # r_final = np.tile(r, (1, self.num_steps))

    return r